In [2]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

In [3]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 42.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

In [5]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

In [6]:
messages = [
    {"role": "system", "content": "use tone as PhD Cambridge professor."},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. This city, distinct from any of the 50 states, serves as the political and administrative heart of the nation. Established by the Constitution of the United States, it houses the three branches of the federal government: the Executive, headed by the President; the Legislative, encompassing Congress; and the Judicial, led by the Supreme Court. Washington, D.C. is also home to numerous national monuments and museums, primarily situated on or around the National Mall.


In [7]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

The expression "3##3" does not have a standard mathematical interpretation. If you are following a specific pattern or notation, please provide more context or details so I can assist you better.


In [8]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


In [10]:
f =open("tweet_collection.txt", "r")
tweets = f.read()
f.close()

In [11]:
tweet_sample = tweets.split("\n")[:10]
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimitered by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_sample}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss various political topics related to an upcoming election. One tweet mentions a donation to Kamala Harris and downballot Democrats. Another highlights the local versus national nature of the election. Concerns about voter fraud are raised, referencing an incident involving Glenn Youngkin's son. Kamala Harris's campaign strategy is questioned for its lack of urgency. Joy Reid from MSNBC is accused of promoting a conspiracy theory about Republicans rigging the election. Billie Jean King emphasizes the importance of the election. There are calls for a weapons embargo as an election strategy. Scott Adams expresses doubt about accepting the election outcome if Trump loses. A user questions if someone will leave if a certain candidate wins. Lastly, a tweet mentions the Chief Election Commissioner's comments on election results in Haryana.


In [12]:
tweet_sample = tweets.split("\n")[:10]
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimitered by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_sample}{delimiter}"},
    ]

print(openai_help(messages))

Tweets focus on election concerns, voter fraud, Kamala Harris's strategy, and conspiracy theories, highlighting political tensions.


In [13]:
tweet_sample = tweets.split("\n")[:10]
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimitered by {delimiter},
                                    focuse on how people discuss about AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_sample}{delimiter}"},
    ]

print(openai_help(messages))

The tweets focus on political discussions, with no direct mention of AI. Conversations revolve around election concerns, voter fraud, and political strategies, highlighting the contentious nature of the upcoming election.


In [14]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)

In [15]:
for tweet in tweets.split('\n')[60:70]:
    flag_help(tweet)

===
violence
===
harassment


In [16]:
tweet_sample = tweets.split("\n")[:10]

for tweet in tweet_sample:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimitered by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

“转发 @MikeNellis: 我说过在选举日之前我会再做一次，所以我们开始吧……\n\n我将捐款给 @KamalaHarris 和其他民主党候选人……”
所有政治都是地方性的吗？这次选举不是这样 https://t.co/rzmTMWE3dc https://t.co/NYCmAYPTb9
“转发 @NotHoodlum: 格伦·杨金对选民欺诈非常关心。然而，当他17岁的儿子被抓到时，他却只字未提……”
"立即阅读：‘她为什么不努力工作’：政治专家质疑哈里斯的策略——哈里斯的低调日程引发政治专家的担忧，他们质疑在距离选举不到三周的情况下，她为何缺乏紧迫感……https://t.co/Mvhejh8Ajo"
“转发 @CollinRugg: 最新消息：MSNBC的Joy Reid推出新的阴谋论，指责*共和党人*操纵选举，称美国已经被…”
RT @BillieJeanKing: 我们距离美国历史上可能最具影响力的选举还有整整3周的时间。 \n\n无论如何强调这一点都不为过…
RT @CallForCongress: 这是一个选举策略\n\n立即实施武器禁运
“转发 @ScottAdamsSays: 如果特朗普在当前条件下输掉选举，我将不接受选举结果的有效性。\n\n我们距离……”
"@GuntherEagleman 我不知道，当她赢得选举时你会离开吗？"
RT @amarDgreat: @sardesairajdeep @ECISVEEP 首席选举专员在新闻发布会上：\n\n在哈里亚纳邦结果公布日：\n我们的计票开始…


In [17]:
tweet_sample = tweets.split("\n")[:10]

for tweet in tweet_sample:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimitered by {delimiter} in the tone of 6 years old kid """},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

"RT @MikeNellis: I'm gonna do this one more time before the big voting day, so here we go... I'm gonna give some of my allowance to @KamalaHarris and all the other people on her team! Yay!"
Politics is like a big game, but this time it's not just in our backyard! Check it out! https://t.co/rzmTMWE3dc https://t.co/NYCmAYPTb9
"RT @NotHoodlum: Glenn Youngkin is super worried about people cheating in voting. But he didn't say anything when his own kid, who's 17, got in trouble for it! That's kinda silly, right?"
"Hey! Why isn't she doing her work? Some grown-ups are wondering why Harris isn't working super hard. They think she should be doing more stuff 'cause there's only a little bit of time left before the big election day! Check it out here: https://t.co/Mvhejh8Ajo"
"RT @CollinRugg: Guess what! Joy Reid from MSNBC is saying something super silly! She thinks the *Republicans* are playing tricks with the election, like when my brother hides my toys! She says America is all mixed up now..

In [18]:
tweet_sample = tweets.split("\n")[:10]

for tweet in tweet_sample:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimitered by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democra or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "positive",
  "emotion": "supportive",
  "mentioned": ["MikeNellis", "KamalaHarris"],
  "support": "Democrat"
}
{
  "sentiment": "neutral",
  "emotion": "indifference",
  "mentioned": [],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "concern",
  "mentioned": ["Glenn Youngkin"],
  "support": "Democrat"
}
{
  "sentiment": "negative",
  "emotion": "concern",
  "mentioned": ["Harris"],
  "support": "Republican"
}
{
  "sentiment": "negative",
  "emotion": "accusatory",
  "mentioned": ["Joy Reid", "Republicans"],
  "support": "Democrat"
}
{
  "sentiment": "neutral",
  "emotion": "anticipation",
  "mentioned": ["BillieJeanKing"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "cynicism",
  "mentioned": ["CallForCongress"],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "distrust",
  "mentioned": ["ScottAdamsSays", "Trump"],
  "support": "Republican"
}
{
  "sentiment": "neutral",
  "emotion": "curiosity",
  "mentione

In [19]:
tweet_sample = tweets.split("\n")[:10]

messages = [
        {"role": "system", "content": f"""analyze the tweet delimitered by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_sample}{delimiter} "}]
print(openai_help(messages))

{
  "topics": [
    "Election Day Donations",
    "Local vs National Politics",
    "Voter Fraud Concerns",
    "Kamala Harris's Campaign Strategy",
    "Election Rigging Accusations",
    "Significance of Upcoming Election",
    "Election Ploys and Strategies",
    "Election Outcome Acceptance",
    "Election Results Speculation",
    "Election Process and Counting"
  ]
}


In [20]:
tweet_sample = tweets.split("\n")[:100]
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_sample):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimitered by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democra or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 100/100 [01:33<00:00,  1.07it/s]


In [21]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis reuslt delimitered by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets that support Democrat and Republican;
                                        step 2 {delimiter} identify the common sentiments and emotoions to each mentioned people;
                                        step 3 {delimiter} organize the result in a json document with keys <Democat count>, <Republican count>, <people name>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "Democrat count": 18,
  "Republican count": 30,
  "Kamala Harris": {
    "common sentiments": ["positive", "negative"],
    "common emotions": ["supportive", "frustration"]
  },
  "Donald Trump": {
    "common sentiments": ["negative", "neutral"],
    "common emotions": ["frustration", "informative"]
  },
  "Biden-Harris": {
    "common sentiments": ["negative"],
    "common emotions": ["anger"]
  },
  "MikeNellis": {
    "common sentiments": ["positive"],
    "common emotions": ["supportive"]
  },
  "Glenn Youngkin": {
    "common sentiments": ["negative"],
    "common emotions": ["concern"]
  },
  "ScottAdamsSays": {
    "common sentiments": ["negative"],
    "common emotions": ["distrust"]
  },
  "Joe Rogan": {
    "common sentiments": ["negative"],
    "common emotions": ["frustration"]
  },
  "MTGrepp": {
    "common sentiments": ["neutral", "negative"],
    "common emotions": ["frustration"]
  },
  "SenJohnKennedy": {
    "common sentiments": ["negative"],
    "common emotion

In [22]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are an election chabot anwser user questions based on the tweets {delimiter} to answer user questions,
                                {delimiter}{tweet_sample}{delimiter} 
                                if user mentioned a people name in the {analysis_summary} people field,report the corresponding sentiment and emotion,
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [23]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

You:  List something.


Chatbot: Could you please specify what you would like me to list?


You:  Exit


Chatbot: Goodbye!
